In [143]:
import pandas as pd
import numpy as np

In [144]:
number_of_subjects = 23
number_of_readings_per_subject = 1001

In [145]:
#generates random data points randomly ranging from 0.5 to 1 and returns an adequate pandas dataset
def generate_random_data(number_of_subjects, points_per_subject):
    dataset = np.zeros((number_of_subjects, points_per_subject))
    for k in range(number_of_subjects):
        for i in range(points_per_subject):
            r_k_i = (np.random.random()+1)/2
            dataset[k][i] = r_k_i

    #set dataset column names (generic)
    data_columns = [f"r_{i}" for i in range(points_per_subject)]
    dataset = pd.DataFrame(dataset, columns = data_columns)
    
    return dataset

In [146]:
dataset = generate_random_data(number_of_subjects, number_of_readings_per_subject)